#### (I). Inference with Phi-3 on 163 HumanEval cases
* Error / Success with Message information included

In [24]:
from sample_hf import CodeCompletionGenerator
from human_eval.data import write_jsonl, read_problems
from tqdm import tqdm

# model_name = "microsoft/Phi-3-mini-128k-instruct"
model_name="meta-llama/Meta-Llama-3-8B"
is_instruct = True
file_name = "eval_llama_3"

code_gen = CodeCompletionGenerator(model_name=model_name, device="mps", is_instruct=is_instruct)

problems = read_problems()
problems = {k: problems[k] for k in list(problems.keys())[:10]} # SubSample Problems out of the 163 cases

num_samples_per_task = 2
total_tasks = len(problems)
pb = tqdm(total=total_tasks * num_samples_per_task, desc="Generating completions")
samples = []
for task_id in problems:
    for _ in range(num_samples_per_task):
        completion = code_gen.generate_one_completion(problems[task_id]["prompt"])
        samples.append(dict(task_id=task_id, completion=completion))
        pb.update(1)
pb.close()
write_jsonl(f"{file_name}.jsonl", samples)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

original/params.json:   0%|          | 0.00/211 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Generating completions: 100%|██████████| 20/20 [01:39<00:00,  4.95s/it]


MLX generation approach is not right, the error rate is way to high ...

In [2]:
from human_eval.data import stream_jsonl
from sample_hf import check_code
from human_eval.data import read_problems
problems = read_problems()



file_name="eval_phi_3_instruct"
sample_file = f"{file_name}.jsonl"
n_success = 0
for sample in stream_jsonl(sample_file): # This is a generator
    task_id = sample['task_id']
    problem = problems[task_id]
    check_program, success, message = check_code(task_id, problem, sample['completion'])
    print("Success: ", success, " || Message: ", message)
    n_success += success

Success:  True  || Message:  
Success:  True  || Message:  
Success:  True  || Message:  
Success:  True  || Message:  
Success:  True  || Message:  
Success:  True  || Message:  
Success:  True  || Message:  
Success:  True  || Message:  
Success:  True  || Message:  
Success:  True  || Message:  
Success:  True  || Message:  
Success:  True  || Message:  
Success:  True  || Message:  
Success:  True  || Message:  
Success:  True  || Message:  
Success:  True  || Message:  
Success:  True  || Message:  
Success:  True  || Message:  
Success:  False  || Message:  Traceback (most recent call last):
  File "<string>", line 38, in <module>
  File "<string>", line 33, in check
  File "<string>", line 15, in rolling_max
IndexError: list index out of range

Success:  False  || Message:  Traceback (most recent call last):
  File "<string>", line 38, in <module>
  File "<string>", line 33, in check
  File "<string>", line 15, in rolling_max
IndexError: list index out of range

Success:  False 

In [26]:
n_success

11